In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import re
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import TensorDataset, DataLoader
from pytorch_pretrained_bert import BertModel, BertTokenizer
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt

# Model and tokenizer

In [ ]:
class BertBinary(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertBinary, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')

        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
    
    def forward(self, tokens, masks=None):
        _, pooled_output = self.bert(tokens, attention_mask=masks, output_all_encoded_layers=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)

        return linear_output

model = BertBinary()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Load and format data

In [ ]:
# Load data 
data = pd.read_csv(r'Data\News_DJIA.csv')

# combine titles 
title_cols = list(data.columns[2:24]) # using Top 22 titles due to BERTs max sequence length of 512
data['News'] = data[title_cols].agg(' '.join, axis = 1)

# remove, quotes and b, from news title cols 
def clean_titles(titles):
    titles = re.sub('b[(\')]','',titles)
    titles = re.sub('b[(\")]','',titles)
    titles = re.sub("\'",'',titles)
    return titles

data['News'] = data.apply(lambda x: clean_titles(x['News']), axis = 1)

# drop un-used cols
data = data.drop(data.columns[2:27], axis = 1)

# Date to datetime object
data['Date'] = pd.to_datetime(data['Date'])

In [ ]:
# split data to train and test
split = dt.datetime(2015,1,1,0,0,0)
train = data[data.Date <= split]
test = data[data.Date > split]
X_train, y_train = np.array(train['News']),np.array(train['Label'])
X_test, y_test = np.array(test['News']),np.array(test['Label'])

# Tokenize, ID, mask, and pad text data

In [ ]:
max_len = 512 # hard limit for BERT

# tokenize text
def tokenize_ID(text):
        
    # tokenize and add clasification and seperation tokens
    text_tokenized = tokenizer.tokenize(text)
    text_tokenized.insert(0,'[CLS]')
    if len(text_tokenized) >= max_len:
        text_tokenized.insert(max_len -1,'[SEP]')
    else:
        text_tokenized.append('[SEP]')
    
    # convert tokens to IDs
    IDs = tokenizer.convert_tokens_to_ids(text_tokenized)

    return IDs

X_train = [tokenize_ID(text) for text in X_train]
X_test = [tokenize_ID(text) for text in X_test]

In [ ]:
# Pad/trim IDs
X_train = pad_sequences(X_train, maxlen=max_len, truncating="post", padding="post")
X_test = pad_sequences(X_test, maxlen=max_len, truncating="post", padding="post")

In [ ]:
# creat masks
train_masks = [[float(i > 0) for i in ii] for ii in X_train]
test_masks = [[float(i > 0) for i in ii] for ii in X_test]

# Train/test

In [ ]:
# Batch data
batch_size = 4

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(X_train),torch.tensor(train_masks), torch.from_numpy(y_train))
test_data = TensorDataset(torch.from_numpy(X_test),torch.tensor(test_masks), torch.from_numpy(y_test))

# create data loaders
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [ ]:
# move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
# training functions and parameters
optimizer = Adam(params = model.parameters(), lr = 1e-4)
loss_func = nn.CrossEntropyLoss(weight=None)
EPOCHS = 4

In [ ]:
for epoch_num in range(EPOCHS):

    model.train()
    train_loss = 0

    # itterate through train data
    for step_num, batch_data in enumerate(train_loader):
        token_ids,masks, labels = tuple(t.to(device) for t in batch_data)

        # https://github.com/huggingface/transformers/issues/2952
        token_ids = token_ids.type(torch.LongTensor)
        labels = labels.type(torch.LongTensor)
        token_ids = token_ids.to(device)
        labels = labels.to(device)
        # end bug fix

        model.zero_grad()
        logits = model(token_ids, masks)  
        batch_loss = loss_func(logits, labels)
        train_loss += batch_loss.item()      
        batch_loss.backward()
        clip_grad_norm_(parameters=model.parameters(), max_norm=1.0)
        optimizer.step()

    model.eval()
    val_loss = 0
    predictions = []
    solutions = []

    # itterate through test data
    with torch.no_grad():
        for step_num_eval, batch_data in enumerate(test_loader):
            token_ids,masks, labels = tuple(t.to(device) for t in batch_data)

            # https://github.com/huggingface/transformers/issues/2952
            token_ids = token_ids.type(torch.LongTensor)
            labels = labels.type(torch.LongTensor)
            token_ids = token_ids.to(device)
            labels = labels.to(device)
            # end bug fix

            logits = model(token_ids, masks)  
            batch_loss = loss_func(logits, labels)
            val_loss += batch_loss.item()

            for pred in logits.cpu().detach().numpy():
                predictions.append(pred.tolist())
            for sol in labels.cpu().detach().numpy():
                solutions.append(sol.item())

        predictions_ = [x.index(max(x)) for x in predictions]
        correct = predictions_ == np.array(solutions)
        accuracy = correct.tolist().count(True)/len(y_test)

    print('Epoch:{}, loss: {}, Accuracy: {}'.format(epoch_num, val_loss/(step_num_eval+1),accuracy))
